In [ ]:
pip install firecrawl

In [3]:
import pandas as pd
import os 
from firecrawl import Firecrawl 
import sys

FIRECREWL_API_KEY = "fc-99ac4e2cc9b64923ac1a0dc507896f04" # คีย์ลับ Firecrawl

# ใช้คีย์ที่กำหนดไว้
API_KEY_TO_USE = FIRECREWL_API_KEY 

try: 
    # 1. Initialize Firecrawl Client
    fc = Firecrawl(api_key=API_KEY_TO_USE)
    
    print(f"INFO: Firecrawl Client Initialized.")

    # 2. เริ่ม Crawl Job
    print("STATUS: Starting crawl job...")
    
    # **************************************************************************
    # การตั้งค่า 'max_discovery_depth': 0 หมายถึง scrape เฉพาะ URL ที่ให้มาเท่านั้น
    result = fc.scrape(
            "https://thestandard.co/spending-grows-despite-card-closures/",
            formats=["markdown"]
    )
    print(result)
 
    
    # 3. ตรวจสอบผลลัพธ์
    if not result:
        raise ValueError("No result returned from Firecrawl.")
    
    # data = result["data"]
    # df = pd.DataFrame(data)
    # print(f"INFO: Found {len(df)} documents.")
    
# 🔹 กรณีผลลัพธ์อยู่ในรูป dictionary ที่มี key 'markdown'
    if isinstance(result, dict) and "markdown" in result:
        markdown = result["markdown"]
    elif isinstance(result, dict) and "data" in result and len(result["data"]) > 0:
        markdown = result["data"][0].get("markdown", "")
    elif isinstance(result, list) and len(result) > 0:
        markdown = result[0].get("markdown", "")
    else:
        markdown = ""

    if not markdown.strip():
        raise ValueError("No markdown content extracted.")
    
    print("\n✅ Successfully scraped content!\n")
    print(markdown[:1000] + "..." if len(markdown) > 1000 else markdown)

except Exception as e:
    print(f"\n❌ Error: {e}")

INFO: Firecrawl Client Initialized.
STATUS: Starting crawl job...
markdown='×\n\n×\n\nHi,\n\n![](https://thestandard.co/spending-grows-despite-card-closures/)\n\n[FOR YOU](https://thestandard.co/membership/#foryou) [HISTORY](https://thestandard.co/membership/#history) [EVENTS](https://thestandard.co/events) [EDIT PROFILE](https://thestandard.co/membership/?mbsaction=Profile_edit) [EDIT CONSENT](https://thestandard.co/membership/?mbsaction=Consent_edit) [LOG OUT](https://thestandard.co/spending-grows-despite-card-closures/#)\n\n[LOG IN](https://thestandard.co/membership/) [SIGN UP](https://thestandard.co/membership/?mbsaction=signup)\n\n![](https://thestandard.co/wp-content/uploads/2020/11/icon-mbs-author-desktop.png?x28583)\n\n![](https://thestandard.co/spending-grows-despite-card-closures/)\n\nHi,\n\nMEMBERSHIP\n\n[FOR YOU](https://thestandard.co/membership/#foryou) [HISTORY](https://thestandard.co/membership/#history) [EVENTS](https://thestandard.co/events) [EDIT PROFILE](https://the

In [ ]:
import re
from firecrawl import Firecrawl
import uuid
from datetime import datetime

FIRECRAWL_API_KEY = "fc-99ac4e2cc9b64923ac1a0dc507896f04"

def clean_article_content(markdown_text):
    """Extract and clean the main article content from scraped markdown."""
    
    # Find the article title and content
    lines = markdown_text.split('\n')
    
    # Variables to store extracted content
    title = ""
    author = ""
    date = ""
    content_lines = []
    in_content = False
    
    for i, line in enumerate(lines):
        # Extract title (usually starts with #)
        if line.startswith('# ') and not title:
            title = line.replace('# ', '').strip()
            in_content = True
            continue
        
        # Extract author (looks for "โดย")
        if 'โดย' in line and not author:
            author = line.strip()
            continue
        
        # Extract date (format: DD.MM.YYYY)
        if re.match(r'\d{2}\.\d{2}\.\d{4}', line.strip()):
            date = line.strip()
            continue
        
        # Start collecting content after basic metadata
        if in_content and line.strip():
            # Stop at common footer markers
            if any(marker in line for marker in [
                'TAGS:', 'ABOUT THE AUTHOR', 'READ MORE', 
                '###', 'สามารถติดตาม', 'TODAY', 'WEEK', 'MONTH'
            ]):
                break
            
            # Skip navigation and UI elements
            if any(skip in line for skip in [
                '![](', '[![', '[LOG IN]', '[SIGN UP]', 
                'MEMBERSHIP', 'FOLLOW US', '* * *'
            ]):
                continue
            
            # Add content lines
            if line.startswith('##') or len(line) > 50 or line.startswith('วันนี้'):
                content_lines.append(line)
    
    # Compile the clean article
    clean_article = f"{title}\n\n"
    if author:
        clean_article += f"{author}\n"
    if date:
        clean_article += f"{date}\n\n"
    clean_article += '\n'.join(content_lines)
    
    return clean_article.strip()

def extract_article_data(markdown_text):
    """Extract structured data from the article."""
    
    article = {}
    
    # Extract title
    title_match = re.search(r'# (.+)', markdown_text)
    article['title'] = title_match.group(1) if title_match else ""
    
    # Extract author
    author_match = re.search(r'โดย \[(.+?)\]', markdown_text)
    article['author'] = author_match.group(1) if author_match else ""
    
    # Extract date
    date_match = re.search(r'(\d{2}\.\d{2}\.\d{4})', markdown_text)
    article['date'] = date_match.group(1) if date_match else ""
    
    # Extract main content paragraphs
    content_start = markdown_text.find('วันนี้')
    content_end = markdown_text.find('### สามารถติดตาม')
    
    if content_start != -1 and content_end != -1:
        main_content = markdown_text[content_start:content_end]
        # Clean up the content
        main_content = re.sub(r'\[.*?\]\(.*?\)', '', main_content)  # Remove links
        main_content = re.sub(r'!\[.*?\]\(.*?\)', '', main_content)  # Remove images
        article['content'] = main_content.strip()
    else:
        article['content'] = ""
    
    return article

# Main execution
try:
    print("INFO: Initializing Firecrawl Client...")
    fc = Firecrawl(api_key=FIRECRAWL_API_KEY)
    
    print("STATUS: Starting scrape job...")
    result = fc.scrape(
        "https://thestandard.co/spending-grows-despite-card-closures/",
        formats=["markdown"]
    )
    
    # Debug: Print result structure
    print(f"\nDEBUG: Result type: {type(result)}")
    
    # Extract markdown content - handle Firecrawl Document object
    markdown = ""
    
    # Try to access as object attribute first (for Document objects)
    if hasattr(result, 'markdown'):
        markdown = result.markdown
        print("DEBUG: Found markdown as object attribute")
    # Then try as dictionary
    elif isinstance(result, dict):
        if "markdown" in result:
            markdown = result["markdown"]
            print("DEBUG: Found markdown in result['markdown']")
        elif "data" in result:
            if isinstance(result["data"], list) and len(result["data"]) > 0:
                markdown = result["data"][0].get("markdown", "")
                print("DEBUG: Found markdown in result['data'][0]['markdown']")
            elif isinstance(result["data"], dict):
                markdown = result["data"].get("markdown", "")
                print("DEBUG: Found markdown in result['data']['markdown']")
    
    if not markdown or not markdown.strip():
        raise ValueError("No markdown content extracted. The document may be empty.")
    
    print("\n" + "="*80)
    print("CLEANED ARTICLE CONTENT")
    print("="*80 + "\n")
    
    # Method 1: Clean article with basic formatting
    clean_content = clean_article_content(markdown)
    print(clean_content)
    
    print("\n" + "="*80)
    print("STRUCTURED DATA EXTRACTION")
    print("="*80 + "\n")
    
    # Method 2: Structured data extraction
    article_data = extract_article_data(markdown)
    print(f"Title: {article_data['title']}")
    print(f"Author: {article_data['author']}")
    print(f"Date: {article_data['date']}")
    print(f"\nContent Preview (first 500 chars):")
    #print(article_data['content'][:] + "..." if len(article_data['content']) > 500 else article_data['content'])
    
    print("\n✅ Article extraction completed successfully!")
    page_id = str(uuid.uuid4())
    now_str = datetime.now().isoformat()

    page_data = {
        "id": "scp_" + page_id,
        "datetime": {article_data['date']},
        "url": result.url,
        "content": result.markdown
    }

    print(type(page_data))

    # ส่งเข้า Meilisearch
    MEILI_URL = "http://10.1.0.150:7700/indexes/web_scraping/documents"

    HEADERS = {"Content-Type": "application/json"
        # "Authorization": "Bearer MASTER_KEY"
                }
    try:
        response = requests.post(MEILI_URL, headers=HEADERS, json=[page_data])
        if response.status_code == 202:
            status = "success"
            message = f"ส่ง Request สำเร็จ!"
        else:
            status = "error"
            message = f"ส่งไม่สำเร็จ ({response.status_code})"
    except Exception as e:
        status = "error"
        message = str(e)

    print({
        "status": status,
        "message": message,
        #"json_content": page_data  # 🔹 แสดงเนื้อไฟล์ JSON
            })

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

INFO: Initializing Firecrawl Client...
STATUS: Starting scrape job...

DEBUG: Result type: <class 'firecrawl.v2.types.Document'>
DEBUG: Found markdown as object attribute

CLEANED ARTICLE CONTENT

กรุงศรี คอนซูมเมอร์ เผยคนปิดบัตรเครดิต มากกว่าเปิดใหม่ ลูกค้าใหม่หดตัว 4.7% แต่ยอดใช้จ่ายยังโตได้ 1%

โดย [โมจัง ลีลา](https://thestandard.co/author/mojung-l/)
06.11.2025

![กรุงศรี คอนซูมเมอร์ เผยคนปิดบัตรเครดิต มากกว่าเปิดใหม่ ลูกค้าใหม่หดตัว 4.7% แต่ยอดใช้จ่ายยังโตได้ 1%](https://thestandard.co/wp-content/uploads/2025/11/spending-grows-despite-card.jpg?x50806)
วันนี้ (6  พ.ย.2568) อธิศ รุจิรวัฒน์ ประธานคณะเจ้าหน้าที่ด้านกรุงศรี คอนซูมเมอร์ผู้ให้บริการด้านบัตรเครดิต และสินเชื่อส่วนบุคคล เปิดเผยว่า ในช่วง 9 เดือนแรก ทุกแบงก์ที่มีการออกบัตรเครดิต มียอดปิดบัญชีบัตรเครดิต มากกว่ายอดเปิดบัญชีใหม่ โดยยอดปิดบัญชี บัตรเครดิตเพิ่มขึ้นในทุกกลุ่มลูกค้า เช่นเดียวกับยอดใช้จ่ายผ่านบัตรที่ลดลง ทั้งนี้เป็นการหดตัวทั้งอุตสาหกรรม ซึ่งมีสาเหตุหลักมาจากการที่ลูกค้าระมัดระวังการใช้จ่าย ลดการก่อหนี้ที่ไม่จำเป็น 